In [1]:
import pandas as pd
import json

In [2]:
def extract_full_name(json_array):
    if isinstance(json_array, list) and len(json_array) > 0:
        main_names = [obj for obj in json_array if obj.get('note') == 'Main']
        main_names = json_array
        if 'name' in main_names[-1]:
            return main_names[-1]['name']
        elif 'honorific_prefix' in main_names[-1] and 'lordname' in main_names[-1] and 'lordofname' in main_names[-1]:
            if main_names[-1]['lordofname'] == '':
                return main_names[-1]['honorific_prefix'] + ' ' + main_names[-1]['lordname']
            else:
                return main_names[-1]['honorific_prefix'] + ' ' + main_names[-1]['lordname'] + ' of ' + main_names[-1]['lordofname']
        elif 'honorific_prefix' in main_names[-1] and 'given_name' in main_names[-1] and 'family_name' in main_names[-1]:
            return main_names[-1]['honorific_prefix'] + ' ' + main_names[-1]['given_name'] + ' ' + main_names[-1]['family_name']
        elif 'given_name' in main_names[-1] and 'lordname' in main_names[-1]:
            return main_names[-1]['given_name'] + ' ' + main_names[-1]['lordname']
        elif 'honorific_prefix' in main_names[-1] and 'family_name' in main_names[-1]:
            return main_names[-1]['honorific_prefix'] + ' ' + main_names[-1]['family_name']
        elif 'given_name' in main_names[-1] and 'family_name' in main_names[-1]:
            return main_names[-1]['given_name'] + ' ' + main_names[-1]['family_name']
        elif 'given_name' in main_names[-1]:
            return main_names[-1]['given_name']
        else:
            return None
    else:
        return None

def redirect_full_name(row):
    if pd.isna(row['speaker_name']):
        match = persons_df[persons_df['id'] == row['redirect']]
        return match.iloc[0]['speaker_name']
    else:
        return row['speaker_name']


In [3]:
speech_df = pd.read_csv('intermediate_outputs/all_speeches.csv', parse_dates=['hdate'])
person_dict = json.load(open('external_datasets/people.json'))

In [4]:
memberships_df = pd.DataFrame(person_dict['memberships'])
memberships_df.rename({
    'id': 'membership_id',
    'on_behalf_of_id': 'party_id',
    'start_date': 'membership_start_date',
    'end_date': 'membership_end_date'
    }, axis = 1, inplace = True)
memberships_df.drop(['end_reason', 'identifiers', 'start_reason', 'label', 'role', 'redirect', 'reason', 'name', 'organization_id'], axis = 1, inplace = True)
memberships_df.dropna(subset=['person_id'], inplace = True)

parties_df = pd.DataFrame(person_dict['organizations'])
parties_df.rename({
    'id': 'party_id',
    'name': 'party_name'
    }, axis = 1, inplace = True)
parties_df.drop(['classification', 'identifiers'], axis = 1, inplace = True)

persons_df = pd.DataFrame(person_dict['persons'])
persons_df['speaker_id'] = persons_df['id'].str.extract(r'(-?\d+)$')
persons_df['speaker_name'] = persons_df['other_names'].apply(lambda x: extract_full_name(x)).tolist()
persons_df['speaker_name'] = persons_df.apply(redirect_full_name, axis=1)
persons_df.rename({'id': 'person_id'}, axis = 1, inplace = True)
persons_df.drop(['identifiers', 'other_names', 'shortcuts', 'redirect'], axis = 1, inplace = True)

posts_df = pd.json_normalize(person_dict['posts'], sep = '_')
posts_df.rename({
    'id': 'post_id',
    'label': 'post_name',
    'area_name': 'post_area_name'
    }, axis = 1, inplace = True)
posts_df.drop(['identifiers', 'organization_id', 'role', 'start_date', 'end_date'], axis = 1, inplace = True)

In [5]:
print('speech_df: ', speech_df.shape)

joined_1 = pd.merge(
    speech_df,
    persons_df,
    how = 'left',
    left_on = 'speaker_id',
    right_on = 'speaker_id'
)

print('joined_1: ', joined_1.shape)

joined_2 = pd.merge(
    joined_1,
    memberships_df,
    how = 'left',
    left_on = 'person_id',
    right_on = 'person_id' # One to many join
)

print('joined_2: ', joined_2.shape)

joined_3 = joined_2[
    (joined_2['person_id'].isna()) |
    (
        (
            (joined_2['hdate'] >=  joined_2['membership_start_date']) |
            (joined_2['membership_start_date'].isna())
        ) &
        (
            (joined_2['hdate'] <= joined_2['membership_end_date']) |
            (joined_2['membership_end_date'].isna())
        )
    )
]

print('joined_3: ', joined_3.shape)

# The shape difference is not due to missing speeches
set_joined_1 = set(joined_1['gid']+joined_1['person_id'])
set_joined_3 = set(joined_3['gid']+joined_3['person_id'])
unique_to_joined_1 = set_joined_1 - set_joined_3
unique_to_joined_3 = set_joined_3 - set_joined_1
print("Unique to joined_1:", unique_to_joined_1)
print("Unique to joined_3:", unique_to_joined_3)

# The shape difference is due to speakers with multiple memberships in the same period. This will be resolved after all joins.
multiple_membership = joined_3[joined_3['speech_body'].duplicated(keep=False)]
multiple_membership = multiple_membership[multiple_membership['gid'].duplicated(keep=False)]
print('multiple_membership: ', multiple_membership.shape)

joined_4 = pd.merge(
    joined_3,
    posts_df,
    how = 'left',
    left_on = 'post_id',
    right_on = 'post_id'
)

print('joined_4: ', joined_4.shape)

joined_5 = pd.merge(
    joined_4,
    parties_df,
    how = 'left',
    left_on = 'party_id',
    right_on = 'party_id'
)

print('joined_5: ', joined_5.shape)


import pandas as pd

# Example DataFrame
# Replace this with your actual DataFrame
speech_person_df = pd.DataFrame({
    # Your columns here
})

def unique_string_agg(series):
    return ', '.join(sorted(set(series.dropna())))

speech_person_df = joined_5.groupby(['gid', 'hdate', 'parent_body', 'file_name', 'html_file_name', 'debate_type', 'written_type', 'speech_body', 'full_url',
       'relevant_speeches', 'speaker_id', 'person_id', 'speaker_name'], as_index = False, dropna = False).agg(unique_string_agg)

print('speech_person_df: ', speech_person_df.shape) # Back to 16436 rows

speech_df:  (16436, 11)
joined_1:  (16436, 13)
joined_2:  (84652, 18)
joined_3:  (16459, 18)
Unique to joined_1: set()
Unique to joined_3: set()
multiple_membership:  (56, 18)
joined_4:  (16459, 20)
joined_5:  (16459, 21)
speech_person_df:  (16436, 21)


In [6]:
speech_person_df.to_csv('all_speeches_and_person.csv', index=False)